In [72]:
import pandas as pd
import numpy as np

In [73]:
df1 = pd.read_excel('hw4_data.xlsx',sheet_name = 'PRICES_DATA')
df2 = pd.read_excel('hw4_data.xlsx', sheet_name='IVV_DATA')
prc = pd.read_csv('prices.csv')

# Rename the columns
df1 = df1.rename(columns={
    'Dates': 'Date',
    'IVV US Equity': 'IVV_US_Equity',
    'IVV AU Equity': 'IVV_AU_Equity',
    'ECRPUS 1Y Index': 'ECRPUS_1Y_Index',
    'SPXSFRCS Index': 'SPXSFRCS_Index',
    'FDTRFTRL Index': 'FDTRFTRL_Index',
    'USCRWTIC Index': 'USCRWTIC_Index',
    'XAU Curncy': 'XAU_Curncy',
    'JPYUSD Curncy': 'JPYUSD_Curncy',
    'DXY Curncy': 'DXY_Curncy',
    'VIX Index': 'VIX_Index'
})
df2 = df2.rename(columns={
    'Dates': 'Date'
})

prc = prc.rename(columns={
    'Open Price': 'open',
    'High Price': 'high',
    'Low Price':'low',
    'Close Price':'close'
})


In [74]:
df1['Date'] = pd.to_datetime(df1['Date'])
df1.loc[78, 'USCRWTIC_Index'] = 1
prc['Date'] = pd.to_datetime(prc['Date'])


    
merged_df = pd.merge(df1, df2, on='Date', how='outer')
merged_df = pd.merge(df1, prc, on='Date', how='outer')
print(f"Number of rows: {merged_df.shape[0]}")
print(f"Number of columns: {merged_df.shape[1]}")
merged_df.drop('Instrument', axis=1, inplace=True)
merged_df.drop('FDTRFTRL_Index', axis=1, inplace=True)

# Dates of features and dates of IVV doesn't 100% match, so drop where IVV data is missing
# Drop the rows where 'close' is null
merged_df.dropna(subset=['close'], inplace=True)
merged_df['Date'] = merged_df['Date'].dt.strftime('%Y-%m-%d')
print(f"After preprocessing, number of rows: {merged_df.shape[0]}")
print(f"After preprocessing, number of columns: {merged_df.shape[1]}")

Number of rows: 857
Number of columns: 17
After preprocessing, number of rows: 826
After preprocessing, number of columns: 15


In [75]:
# Find the rows where any column contains null values
null_rows = merged_df[merged_df.isnull().any(axis=1)]

# Print the null rows
null_rows

,Date,IVV_US_Equity,IVV_AU_Equity,ECRPUS_1Y_Index,SPXSFRCS_Index,USCRWTIC_Index,XAU_Curncy,JPYUSD_Curncy,DXY_Curncy,VIX_Index,IVV_AU_split,open,high,low,close


In [76]:
merged_df

,Date,IVV_US_Equity,IVV_AU_Equity,ECRPUS_1Y_Index,SPXSFRCS_Index,USCRWTIC_Index,XAU_Curncy,JPYUSD_Curncy,DXY_Curncy,VIX_Index,IVV_AU_split,open,high,low,close
1,2020-01-02,326.32,462.93,30.0,3318,61.18,1529.13,0.009211,96.846,12.47,1.0,324.98,326.320,323.950,326.32
2,2020-01-03,323.81,464.92,30.0,3318,63.05,1552.20,0.009250,96.838,14.02,1.0,322.53,325.060,322.511,323.81
3,2020-01-06,325.09,465.50,30.0,3318,63.27,1565.74,0.009228,96.671,13.85,1.0,321.89,325.140,321.780,325.09
4,2020-01-07,324.20,469.82,30.0,3318,62.70,1574.37,0.009222,97.005,13.79,1.0,324.46,324.970,323.650,324.20
5,2020-01-08,325.85,470.25,30.0,3318,59.61,1556.42,0.009164,97.299,13.45,1.0,324.38,327.210,324.110,325.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,2023-04-06,411.01,40.67,65.0,4050,80.70,2007.91,0.007588,101.822,18.40,1.0,408.57,411.265,407.440,411.01
853,2023-04-10,411.48,40.67,65.0,4050,79.74,1991.48,0.007485,102.578,18.97,1.0,408.37,411.500,407.760,411.48
854,2023-04-11,411.47,41.07,65.0,4050,81.53,2003.62,0.007480,102.204,19.10,1.0,412.02,412.970,410.690,411.47
855,2023-04-12,409.93,41.13,65.0,4050,83.26,2014.93,0.007511,101.500,19.09,1.0,413.68,413.940,409.225,409.93


In [77]:
# Loop over the columns of df1 except for 'IVV_US_Equity'
exc_list = ['IVV_US_Equity','IVV_AU_split','Date']
for col in merged_df.columns:
    if col not in exc_list:
        merged_df[f'{col}_log_return'] = np.log(merged_df[col] / merged_df[col].shift(1))
        print(col)
        


df1['AU_log_return'] = np.log(df1['IVV_AU_Equity'] *(1/df1['IVV_AU_split'])/ df1['IVV_AU_Equity'].shift(1))

IVV_AU_Equity
ECRPUS_1Y_Index
SPXSFRCS_Index
USCRWTIC_Index
XAU_Curncy
JPYUSD_Curncy
DXY_Curncy
VIX_Index
open
high
low
close


In [78]:
# Find the rows where any column contains null values
null_rows = merged_df[merged_df.isnull().any(axis=1)]

# Print the null rows
null_rows

,Date,IVV_US_Equity,IVV_AU_Equity,ECRPUS_1Y_Index,SPXSFRCS_Index,USCRWTIC_Index,XAU_Curncy,JPYUSD_Curncy,DXY_Curncy,VIX_Index,...,SPXSFRCS_Index_log_return,USCRWTIC_Index_log_return,XAU_Curncy_log_return,JPYUSD_Curncy_log_return,DXY_Curncy_log_return,VIX_Index_log_return,open_log_return,high_log_return,low_log_return,close_log_return
1,2020-01-02,326.32,462.93,30.0,3318,61.18,1529.13,0.009211,96.846,12.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
merged_df.to_excel("merged_df.xlsx")

In [81]:
# Access the row at index 767
row = merged_df.iloc[78]

# Print the row
print(row)

Date                          2020-04-24
IVV_US_Equity                      283.9
IVV_AU_Equity                      437.4
ECRPUS_1Y_Index                    100.0
SPXSFRCS_Index                      2960
USCRWTIC_Index                     16.04
XAU_Curncy                        1729.6
JPYUSD_Curncy                   0.009302
DXY_Curncy                        100.38
VIX_Index                          35.93
IVV_AU_split                         1.0
open                               281.6
high                              284.58
low                               279.39
close                              283.9
IVV_AU_Equity_log_return        -0.00967
ECRPUS_1Y_Index_log_return           0.0
SPXSFRCS_Index_log_return            0.0
USCRWTIC_Index_log_return       0.018245
XAU_Curncy_log_return          -0.000526
JPYUSD_Curncy_log_return        0.001076
DXY_Curncy_log_return          -0.000528
VIX_Index_log_return           -0.141225
open_log_return                 0.000888
high_log_return 